In [3]:
import pandas as pd
from database import engine_maker
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
engine = engine_maker(os.getenv('BRONZE_USER'),os.getenv('BRONZE_PASSWORD'),os.getenv('BRONZE_PORT'),os.getenv('BRONZE_DB'))

In [5]:
sales = pd.read_sql('sales',engine)
customers = pd.read_sql('customers',engine)
products = pd.read_sql('products',engine)

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
#####      Cleaning        ###### 
##### I'll first deal with CustomerNo ==> integer instead of float
##### Then I'll deal with Null Values 
##### And I'll clean the country coulumn 

In [41]:
sales.head()

,index,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,date_shippped,status,Country_Noise
0,0,581482,12/9/2019,22485,21.47,12,17490.0,None,Cancelled,united kingdom
1,1,581475,12/9/2019,22596,10.65,36,13069.0,None,Pending,%United Kingdom
2,2,581475,12/9/2019,23235,11.53,12,13069.0,2019-12-26,Completed,United Kinnngdom
3,3,581475,12/9/2019,23272,10.65,12,13069.0,None,Pending,UUUUnited Kingdom
4,4,581475,12/9/2019,23239,11.94,6,13069.0,None,Cancelled,United K!ingdom


In [42]:
sales.isnull().any()

index            False
TransactionNo    False
Date             False
ProductNo        False
Price            False
Quantity         False
CustomerNo        True
date_shippped     True
status           False
Country_Noise    False
dtype: bool

In [43]:
sales['CustomerNo'] = pd.to_numeric(sales['CustomerNo'], errors='coerce').astype('Int64')

In [44]:
sales.dtypes

index              int64
TransactionNo     object
Date              object
ProductNo         object
Price            float64
Quantity           int64
CustomerNo         Int64
date_shippped     object
status            object
Country_Noise     object
dtype: object

In [45]:
sales['CustomerNo'].head()

0    17490
1    13069
2    13069
3    13069
4    13069
Name: CustomerNo, dtype: Int64

In [46]:
### 0 ==> unknown ####
sales['CustomerNo'] = sales['CustomerNo'].fillna(0)

In [47]:
sales['CustomerNo'].sort_values(ascending=False).unique()

<IntegerArray>
[18287, 18283, 18282, 18281, 18280, 18278, 18277, 18276, 18274, 18273,
 ...
 12042, 12031, 12026, 12025, 12024, 12013, 12008, 12006, 12004,     0]
Length: 4739, dtype: Int64

In [48]:
sales['date_shippped'] = sales['date_shippped'].fillna(1999-12-31)

In [49]:
sales['date_shipped'] = pd.to_datetime(sales['date_shippped'],errors='coerce')

In [51]:
sales['date_shipped'].min()

Timestamp('1970-01-01 00:00:00.000001956')

index            False
TransactionNo    False
Date             False
ProductNo        False
Price            False
Quantity         False
CustomerNo       False
date_shippped    False
status           False
Country_Noise    False
date_shipped     False
dtype: bool

In [1]:
#### first let's get rid of special caracters
#sales['Country_Noise'].str.replace(r'\s+',' ').str.lower().str.strip().str.replace(r'[*,%@-/!$£]+','')
sales['cleaned_country'] = (sales['Country_Noise']
    .str.lower()
    .str.replace(r'[^a-z\s]', '', regex=True)
    .str.replace(r'\s+',' ',regex=True)
    .str.strip()
 )

NameError: name 'sales' is not defined

MemoryError: Unable to allocate 1023. MiB for an array with shape (250, 536350) and data type float64